In [15]:
%load_ext autoreload
%autoreload 2

import os
import tempfile

%matplotlib inline
import matplotlib.pyplot as plt
import pprint
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")
print(type(ratings), type(movies))
for i in ratings.take(2).as_numpy_iterator():
    pprint.pprint(i)
for i in movies.take(2).as_numpy_iterator():
    pprint.pprint(i)

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'> <class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>
{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}
{'bucketized_user_age': 25.0,
 'movie_genres': array([ 4, 14]),
 'movie_id': b'709',
 'movie_title': b'Strictly Ballroom (1992)',
 'raw_user_age': 32.0,
 'timestamp': 875654590,
 'user_gender': True,
 'user_id': b'92',
 'user_occupation_label': 5,
 'user_occupation_text': b'entertainment',
 'user_rating': 2.0,
 'user_zip_code': b'80525'}
{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}
{'movie_genres': array([4, 7]),
 'movie_id': b'1457',
 'movie_title': b'Love Is All There Is (19

2021-09-08 18:12:23.641709: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2021-09-08 18:12:23.676380: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [17]:
numbers = tf.ragged.constant([[8, 9, 3, 4], [], [1, 0], [-6], []])
print(numbers)

<tf.RaggedTensor [[8, 9, 3, 4], [], [1, 0], [-6], []]>


In [53]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

#maxlen = 5
#padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(ratings.map(lambda x: x["movie_genres"]), maxlen=maxlen, padding="post")
#padded_inputs = tf.data.Dataset.from_tensor_slices({"padded_genres": padded_inputs})

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
    "movie_id": x["movie_id"],
    "movie_genres": x["movie_genres"]
})

movies = movies.map(lambda x: {
    "movie_title": x["movie_title"],
    "movie_id": x["movie_id"],
    "movie_genres": x["movie_genres"]
})

In [41]:
for i in ratings.take(2).as_numpy_iterator():
    pprint.pprint(i)
for i in movies.take(2).as_numpy_iterator():
    pprint.pprint(i)

{'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'timestamp': 879024327,
 'user_id': b'138'}
{'movie_genres': array([ 4, 14]),
 'movie_id': b'709',
 'movie_title': b'Strictly Ballroom (1992)',
 'timestamp': 875654590,
 'user_id': b'92'}
{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}
{'movie_genres': array([4, 7]),
 'movie_id': b'1457',
 'movie_title': b'Love Is All There Is (1996)'}


2021-09-08 17:51:14.722316: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2021-09-08 17:51:14.773329: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
a = movies.map(lambda x: x["movie_genres"])
for i in a.take(5).as_numpy_iterator():
    print(i, i.shape, i[0], type(i[0]))

### Reconstruct the tf dataset

In [ ]:
ratings_list = list(ratings.as_numpy_iterator())
movies_list = list(movies.as_numpy_iterator())

In [60]:
print(len(ratings_list), ratings_list[:2])
print(len(movies_list), movies_list[:2])

100000 [{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)", 'user_id': b'138', 'timestamp': 879024327, 'movie_id': b'357', 'movie_genres': array([7])}, {'movie_title': b'Strictly Ballroom (1992)', 'user_id': b'92', 'timestamp': 875654590, 'movie_id': b'709', 'movie_genres': array([ 4, 14])}]
1682 [{'movie_title': b'You So Crazy (1994)', 'movie_id': b'1681', 'movie_genres': array([4])}, {'movie_title': b'Love Is All There Is (1996)', 'movie_id': b'1457', 'movie_genres': array([4, 7])}]


In [120]:
rating_keys = ['movie_title', 'user_id', 'timestamp', 'movie_id', 'movie_genres']
movie_keys = ['movie_title', 'movie_id', 'movie_genres']

In [121]:
new_ratings = {j: [i[j] for i in ratings_list] for j in rating_keys}
new_movies = {j: [i[j] for i in movies_list] for j in movie_keys}

In [122]:
new_ratings['movie_genres'] = [','.join([str(j) for j in list(i)]) for i in new_ratings['movie_genres']]
new_movies['movie_genres'] = [','.join([str(j) for j in list(i)]) for i in new_movies['movie_genres']]

In [125]:
ratings = tf.data.Dataset.from_tensor_slices({'movie_title': new_ratings['movie_title'], 
                                              'user_id': new_ratings['user_id'], 
                                              'timestamp': new_ratings['timestamp'], 
                                              'movie_id': new_ratings['movie_id'],
                                             'movie_genres': new_ratings['movie_genres']})
movies = tf.data.Dataset.from_tensor_slices(new_movies)

In [128]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_movie_titles = np.unique(np.concatenate(list(movies.map(lambda x: x['movie_title']).batch(1000))))
unique_movie_ids = np.unique(np.concatenate(list(movies.map(lambda x: x['movie_id']).batch(1000))))
# unique_movie_genres = np.unique(np.concatenate(list(movies.map(lambda x: x["movie_genres"]))))
unique_user_ids = np.unique(np.concatenate(list(ratings.map(lambda x: x["user_id"]).batch(1000))))

#unique_movie_genres = np.unique(np.concatenate(list(movies.map(lambda x: x["movie_genres"]))))
#unique_movie_genres

In [130]:
a = movies.map(lambda x: x["movie_genres"])
for i in a.take(5).as_numpy_iterator():
    print(i)

b'4'
b'4,7'
b'1,3'
b'0'
b'7'


### Query model

In [131]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])
    self.timestamp_embedding = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.Discretization(timestamp_buckets.tolist()),
        tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
    ])
    self.normalized_timestamp = tf.keras.layers.experimental.preprocessing.Normalization(axis=None)
    self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        #self.normalized_timestamp(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    ], axis=1)

In [132]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.
    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
        self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
        self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    """
    Inputs embedding and go through the dense layers
    Arguments:
      -- inputs: tf dictionary - tensor dictionary contains multiple features of users
    Return:
      -- dense_layers(feature_embedding): tf tensor - values of dense layers
    """
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

### Candidate model

In [11]:
title_text = tf.keras.layers.experimental.preprocessing.TextVectorization()
title_text.adapt(ratings.map(lambda x: x["movie_title"]))

In [14]:
for row in ratings.batch(1).map(lambda x: x["movie_title"]).take(10):
  print(row, title_text(row))

tf.Tensor([b"One Flew Over the Cuckoo's Nest (1975)"], shape=(1,), dtype=string) tf.Tensor([[1 1 1 2 1 1 1]], shape=(1, 7), dtype=int64)
tf.Tensor([b'Strictly Ballroom (1992)'], shape=(1,), dtype=string) tf.Tensor([[1 1 1]], shape=(1, 3), dtype=int64)
tf.Tensor([b'Very Brady Sequel, A (1996)'], shape=(1,), dtype=string) tf.Tensor([[1 1 1 1 1]], shape=(1, 5), dtype=int64)
tf.Tensor([b'Pulp Fiction (1994)'], shape=(1,), dtype=string) tf.Tensor([[1 1 1]], shape=(1, 3), dtype=int64)
tf.Tensor([b'Scream 2 (1997)'], shape=(1,), dtype=string) tf.Tensor([[1 1 1]], shape=(1, 3), dtype=int64)
tf.Tensor([b'Crash (1996)'], shape=(1,), dtype=string) tf.Tensor([[1 1]], shape=(1, 2), dtype=int64)
tf.Tensor([b'Aladdin (1992)'], shape=(1,), dtype=string) tf.Tensor([[1 1]], shape=(1, 2), dtype=int64)
tf.Tensor([b'True Romance (1993)'], shape=(1,), dtype=string) tf.Tensor([[1 1 1]], shape=(1, 3), dtype=int64)
tf.Tensor([b'Bob Roberts (1992)'], shape=(1,), dtype=string) tf.Tensor([[1 1 1]], shape=(1, 3), 

In [133]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000
    
    # movie_
    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ]) # embedding has no input_length

    
    self.title_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_tokens)
    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])
    self.title_vectorizer.adapt(movies.map(lambda x: x['movie_title']))
    

    self.id_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
          vocabulary=unique_movie_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_ids) + 1, 32)
    ])
    
    """ array of genres """ 
    #genre_num = 5
    #self.genres_id_lookup = tf.keras.layers.experimental.preprocessing.IntegerLookup()
    #self.genres_embedding = tf.keras.Sequential([
      #self.genres_id_lookup,
      #tf.keras.layers.Embedding(len(unique_movie_genres)+1, 32, mask_zero=True),
      #tf.keras.layers.GlobalAveragePooling1D(),
    #])
    #self.genres_id_lookup.adapt(movies.map(lambda x: x["movie_genres"]))
    
    max_genres = 22
    self.genres_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(max_tokens=max_genres)
    self.genres_embedding = tf.keras.Sequential([
      self.genres_vectorizer,
      tf.keras.layers.Embedding(max_genres, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])
    self.genres_vectorizer.adapt(movies.map(lambda x: x['movie_title']))
    

  def call(self, titles):
    """
    32-dimension embeddings of each feature are concatenated (32 X number of features)
    return:
      -- Concatenated (32 X number of features)
    """
    return tf.concat([
        self.title_embedding(titles['movie_title']),
        self.title_text_embedding(titles['movie_title']),
        self.id_embedding(titles['movie_id']),
        self.genres_embedding(titles['movie_genres']),
    ], axis=1)

In [143]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = MovieModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
        self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
        self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    """
    Inputs embedding and go through the dense layers
    Arguments:
      -- inputs: tf dictionary - tensor dictionary contains multiple features of movies
    Return:
      -- dense_layers(feature_embedding): tf tensor - values of dense layers
    """
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)
    # return feature_embedding

### Combined model

In [135]:
layer_sizes = [36]
candidate_model = CandidateModel(layer_sizes)
candidates = movies.batch(128).map(candidate_model)
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(candidate_model),),)

In [136]:
for i in candidates.take(2).as_numpy_iterator():
    print(len(i), len(i[0]), i[0].shape)
    print(i)

128 128 (128,)
[[-0.01146629  0.0494458  -0.0304136  ...  0.00677372  0.00883717
   0.0170493 ]
 [-0.04056208 -0.014879    0.01497013 ...  0.00677372  0.00883717
   0.0170493 ]
 [-0.0006185   0.02809947 -0.00539805 ...  0.00677372  0.00883717
   0.0170493 ]
 ...
 [-0.00436002 -0.0043005  -0.01528854 ...  0.00677372  0.00883717
   0.0170493 ]
 [-0.03700734  0.01654127 -0.04621185 ...  0.00677372  0.00883717
   0.0170493 ]
 [-0.01145233  0.00814867  0.04739863 ...  0.00677372  0.00883717
   0.0170493 ]]
128 128 (128,)
[[-0.02472454 -0.00952109  0.0455688  ...  0.00677372  0.00883717
   0.0170493 ]
 [ 0.03062434 -0.04533291  0.03164179 ...  0.00677372  0.00883717
   0.0170493 ]
 [ 0.01599726 -0.03422933 -0.04874967 ...  0.00677372  0.00883717
   0.0170493 ]
 ...
 [-0.01026998 -0.00755505 -0.04873674 ...  0.00677372  0.00883717
   0.0170493 ]
 [-0.00827784  0.01520301 -0.00301795 ...  0.00677372  0.00883717
   0.0170493 ]
 [-0.03011192 -0.04462643 -0.0056051  ...  0.00677372  0.00883717
  

In [144]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
    })
    movie_embeddings = self.candidate_model({
        "movie_id": features["movie_id"],
        "movie_title": features["movie_title"],
        "movie_genres": features["movie_genres"],
    })     # """ This is where the problem is """

    return self.task(query_embeddings, movie_embeddings, compute_metrics=not training)

In [139]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [140]:
for i in ratings.take(2).as_numpy_iterator():
    pprint.pprint(i)

{'movie_genres': b'7',
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'timestamp': 879024327,
 'user_id': b'138'}
{'movie_genres': b'4,14',
 'movie_id': b'709',
 'movie_title': b'Strictly Ballroom (1992)',
 'timestamp': 875654590,
 'user_id': b'92'}


In [141]:
for i in cached_train.take(2).as_numpy_iterator():
    pprint.pprint(i)

{'movie_genres': array([b'0,5,7', b'1,3,8', b'5,7,16', ..., b'7', b'4', b'10,16'],
      dtype=object),
 'movie_id': array([b'127', b'1133', b'100', ..., b'366', b'367', b'288'], dtype=object),
 'movie_title': array([b'Godfather, The (1972)', b'Escape to Witch Mountain (1975)',
       b'Fargo (1996)', ..., b'Dangerous Minds (1995)',
       b'Clueless (1995)', b'Scream (1996)'], dtype=object),
 'timestamp': array([880859493, 882386848, 879442537, ..., 891310875, 892836551,
       889388862], dtype=int32),
 'user_id': array([b'424', b'429', b'53', ..., b'766', b'125', b'478'], dtype=object)}
{'movie_genres': array([b'13,16', b'12,14', b'0,14,16', ..., b'5,7,14,16', b'5,7,9', b'4'],
      dtype=object),
 'movie_id': array([b'479', b'705', b'748', ..., b'129', b'299', b'368'], dtype=object),
 'movie_title': array([b'Vertigo (1958)', b"Singin' in the Rain (1952)",
       b'Saint, The (1997)', ..., b'Bound (1996)', b'Hoodlum (1997)',
       b'Bio-Dome (1996)'], dtype=object),
 'timestamp': a

In [145]:
num_epochs = 20

model = MovielensModel([64,32])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

one_layer_history = model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=2,
    epochs=num_epochs,
    verbose=1)

accuracy = one_layer_history.history["val_factorized_top_k/top_100_categorical_accuracy"][-1]
print(f"Top-100 accuracy: {accuracy:.2f}.")

model.evaluate(cached_test, return_dict=True)

Epoch 1/20
40/40 [==============================] - 3s 42ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 15162.7701 - regularization_loss: 0.0000e+00 - total_loss: 15162.7701
Epoch 2/20
40/40 [==============================] - 5s 114ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 14516.4145 - regularization_loss: 0.0000e+00 - total_loss: 14516.4145 - val_factorized_top_k/top_1_categorical_accuracy: 5.5000e-04 - val_factorized_top_k/top_5_categorical_accuracy: 0.0031 - val

{'factorized_top_k/top_1_categorical_accuracy': 0.0010999999940395355,
 'factorized_top_k/top_5_categorical_accuracy': 0.007699999958276749,
 'factorized_top_k/top_10_categorical_accuracy': 0.017899999395012856,
 'factorized_top_k/top_50_categorical_accuracy': 0.12030000239610672,
 'factorized_top_k/top_100_categorical_accuracy': 0.24619999527931213,
 'loss': 27945.921875,
 'regularization_loss': 0,
 'total_loss': 27945.921875}